# Final Project

Mandatory!

- try, except (Error Handling)
- Class OOP

Optional but require at least 1

- Decorators
- iterator and generators
- Abstract Class
- Context Manager
- String Encodings


# Python 2 HSUTCC: Session 13 Context Manager


# Context Managers


In [ ]:
with open('sameple.txt', 'a+') as f:
    f.read()

### Why using context managers?


Context managers allow to express in a compact form a pattern which we are already familiar with to
work with resources:

```python
r = acquire_resource()
try:
    do_something(r)
finally:
    release_resource(r)
```

With a context manager the example above can be rewritten like this:

```python
with acquire_resource() as r:
    do_something(r)
```

Action `release_resource()` will be invoked automatically, you don’t need to call it explicitely


> Context managers in Python (using the with statement) are a clean way to handle resource management, especially for things that need proper setup and cleanup (like files, locks, database connections etc.)


In [ ]:
class MyIterator:
    def __init__(self):
        pass

    def __iter__(self):
        return

    def __next__(self):
        return

### Context managers’ protocol

A context managers’ protocol consists of two methods.

- A method `__enter__` initializes a context, for example, opens a file or acquire a mutex. A
  value which `__enter__` returns is written by name which is specified after `as` operator.
- A method `__exit__` is called after the execution of the body of `with` operator. The method
  takes three arguments: 1. a type of exception, 2. the exception itself and 3. an object of type traceback. If during the execution of the body of `with` operator there was invoked an exception the `__exit__` method can suppress it and return `True`.

> An instance of any class which implements these two methods is a context manager.


### “Semantics” of `with` operator


Let me remind you that to use any context manager, it should be in the form of

```python
with acquire_resource() as r:
    do_something(r)
```

Therefore, a process of with operator execution can be conceptually written as:

```python
# 1. Create the context manager object
manager = acquire_resource()

# 2. Enter the context, which might set up resources
r = manager.__enter__()

try:
    # 3. Run your code using the resource
    do_something(r)

finally:
    # 4. Get information about any exceptions that occurred
    exc_type, exc_value, tb = sys.exc_info()

    # 5. Exit the context, passing any exception info
    suppress = manager.__exit__(exc_type, exc_value, tb)

    # 6. If there was an exception and the context manager
    # didn't handle it (suppress=False), re-raise it
    if exc_value is not None and not suppress:
        raise exc_value
```


You can have nested context managers at the same time:

```python
with acquire_resource() as r:
    with acquire_other_resource() as other:
        do_something(r, other)
```


## One obvious example


`open()` function returns `FileIO` (`TextIOWrapper`, more specifically) instance whose class is inherited from `_IOBase` which does implement \_\_enter\_\_() and \_\_exit\_\_() methods.


In [1]:
f = open('test.txt', 'w')
print(type(f))  # <class '_io.TextIOWrapper'>

print(hasattr(f, '__enter__'))
print(hasattr(f, '__exit__'))

<class '_io.TextIOWrapper'>
True
True


The class `_IOBase` has implemented `__enter__()` and `__exit__()` methods like this:

```python
class _IOBase:
    def __enter__(self):  # Called when entering 'with' block
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):  # Called when exiting 'with' block
        try:
            self.close()
        finally:
            return None
```

Here, it means that when exiting the context manager, `__exit__()` is called and the file **automatically** closed.


Therefore, you can use `open()` with context manager like this.


In [ ]:
with open('sample.txt', 'r') as test_file:  # test_file = open('test.txt' ,'r')
    print(test_file.tell())
    test_file.seek(0)

test_file.read()
# You do not need to close it anymore, because context manager has already done that for you.

0


ValueError: I/O operation on closed file.

In [8]:
f = open('sample.txt', 'r')
f.close()

Or open two files at the same time:


In [ ]:
with open('test.txt', 'w') as test_file:
    with open('taylor_swift.txt', 'r') as taylor_file:
        first_line = taylor_file.readline()
        test_file.write(first_line)

# You do not need to close it anymore, because context manager has already done that for you.

In [ ]:
def square(x):
    2/0
    return x ** 2


def solve_quadratic_equation(a, b, c):
    return (-b + (square(b) - 4 * a * c))/(2*a)


solve_quadratic_equation(1, 2, 5)

ZeroDivisionError: division by zero

## Creating Your Own Context Manager


In [34]:
class MyContextManager:
    def __init__(self, dummy: str):
        self.dummy = dummy

    def __enter__(self):
        print(f"Entering context with {self.dummy}")
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        if not exc_type:
            print("Exiting context without ERROR")
        if exc_type == ZeroDivisionError:
            print('ZeroDivisionError. Abort')
            print(exc_type, exc_val, exc_tb, sep="\n")
            return True  # (suppress) Suppress the error
        if exc_type == NameError:
            print('WOW')
            return False

        return None

In [ ]:
with MyContextManager("Vetit") as v:
    x + 2
    2/0

Entering context with Vetit
WOW


NameError: name 'x' is not defined

In [ ]:
with MyContextManager("Vetit") as cm:
    x + 2
    # pass

Entering context with Vetit
WOW


In [44]:
import time

t1 = time.time()
99999**99999
t2 = time.time()

t2-t1

0.09042215347290039

## In class task


Implement a `Timer` class which should be a context manager that

1. `__enter__` method should record the time when started (this method should always returns resource, or more specifically, `self`)
2. `__exit__` method should record the ending time and prints out total time taken between `__enter__` and `__exit__`
   You may use module `time` to record the time in ms.

Usage:

```python
with Timer() as t:
    fibonacci(30) # This will print the time taken to calculate fibonacci index 30
```


In [14]:
import time


class Timer:
    def __enter__(self):
        self.time = time.time()

    def __exit__(self, exc_type, exc_val, exc_tb):
        if not exc_type:
            end_time = time.time()
            delta_time = end_time - self.time
            print(delta_time, 'seconds')
        if exc_type == NameError:
            return False


def fibo(n):
    sequence = []
    for i in range(n):
        if len(sequence) < 2:
            sequence.append(1)
        else:
            sequence.append(sequence[i - 1] + sequence[i - 2])
    return sequence

# def fibo(n):
#     if n == 1 or n == 2:
#         return 1
#     else:
#         return fibo(n - 1) + fibo(n - 2)


with Timer() as t:
    fibo(35)

7.867813110351562e-06 seconds


---


# Task


### Create a Custom Console Logger

Create a context manager called `ConsoleLogger` that:

1. Redirects all `print` statements within its context to a file
2. Restores normal console output when exiting the context
3. Handles any exceptions that occur and logs them to the file
4. Provides a summary of how many lines were written when exiting

Requirements:

- Use a class-based approach implementing **enter** and **exit**
- The file name should be provided when creating the context manager
- All prints within the context should go to the file instead of console
- Handle at least 2 exception errors, log them to the file instead of showing
- When exiting, print a summary to the console showing how many lines were logged

Example usage should look like this:

```python
with ConsoleLogger('output.log'):
    print("This should go to the file")
    print("This too")
    # If any error occurs, it should be logged to the file
    x = 2/0 # This should log with 'division by zero' message or something similar.
```

Inside the `output.log`:

```txt
This should go to the file
This too
zero division error
```


In [ ]:
class ConsoleLogger:
    def __init__(self, filename):
        self.filename = filename
        self.file_handle = None
        self.original_print = __builtins__.print

    def __enter__(self):
        self.file_handle = open(self.filename, 'w')

        def custom_print(*args):
            text = ' '.join(str(arg) for arg in args)
            self.file_handle.write(text + '\n')

        __builtins__.print = custom_print
        return self

    def __exit__(self, exc_type, exc_value, exc_tb):
        __builtins__.print = self.original_print

        if self.file_handle:
            self.file_handle.close()

        if exc_type is not None:
            with open(self.filename, 'a') as f:
                if exc_type == ZeroDivisionError:
                    f.write("zero division error\n")
                elif exc_type == NameError:
                    f.write("name error\n")
        return True


print("This goes to console normally")

with ConsoleLogger('session13.log'):
    print("This should go to the file")
    print("This too")
    x = 2/0

print("Back to console output")

This goes to console normally
Back to console output
